In [ ]:
!pip install catboost -q

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import warnings
import random
import os
plt.style.use('ggplot')
warnings.filterwarnings(action='ignore')

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정
# 2023 35 7
# 20000 36 6

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/train.csv')
test = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/test.csv')

In [ ]:
# train = train.drop(columns=['X_124', 'X_125','X_126', 'X_127'])
# test = test.drop(columns=['X_124', 'X_125','X_126', 'X_127'])

In [ ]:
col_list = train.columns
nan_list = []
nan_cnt = []
nan_col = []
full_list = []
for col in col_list:
    if train[col].isnull().sum() == 0 :
        full_list.append(col)
        continue
    nan_list.append([col, train[col].isnull().sum()])
    nan_cnt.append(train[col].isnull().sum())
    nan_col.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_list :
    if col[1] == 598 :
        del_col.append(col[0])
train = train.drop(columns=del_col)
test = test.drop(columns=del_col)
train.head(3)

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN


In [ ]:
trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

In [ ]:
col_list = train.columns
nan_listA_31 = []
nan_cntA_31 = []
nan_colA_31 = []
full_listA_31 = []
for col in col_list:
    if trainA_31[col].isnull().sum() == 0 :
        full_listA_31.append(col)
        continue
    nan_listA_31.append([col, trainA_31[col].isnull().sum()])
    nan_cntA_31.append(trainA_31[col].isnull().sum())
    nan_colA_31.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listA_31 :
    if col[1] == len(trainA_31) :
        del_col.append(col[0])
trainA_31 = trainA_31.drop(columns=del_col)
testA_31 = testA_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = trainA_31.columns
for col in col_list[6:] :
    if trainA_31[col].nunique()==1 :
        del_col.append(col)
trainA_31 = trainA_31.drop(columns=del_col)
testA_31 = testA_31.drop(columns=del_col)

In [ ]:
col_list = train.columns
nan_listO = []
nan_cntO = []
nan_colO = []
full_listO = []
for col in col_list:
    if train_O_31[col].isnull().sum() == 0 :
        full_listO.append(col)
        continue
    nan_listO.append([col, train_O_31[col].isnull().sum()])
    nan_cntO.append(train_O_31[col].isnull().sum())
    nan_colO.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listO :
    if col[1] == len(train_O_31) :
        del_col.append(col[0])
train_O_31 = train_O_31.drop(columns=del_col)
test_O_31 = test_O_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_O_31.columns
for col in col_list[6:] :
    if train_O_31[col].nunique()==1 :
        del_col.append(col)
train_O_31 = train_O_31.drop(columns=del_col)
test_O_31 = test_O_31.drop(columns=del_col)

In [ ]:
col_list = train.columns
nan_listT = []
nan_cntT = []
nan_colT = []
full_listT = []
for col in col_list:
    if train_T_31[col].isnull().sum() == 0 :
        full_listT.append(col)
        continue
    nan_listT.append([col, train_T_31[col].isnull().sum()])
    nan_cntT.append(train_T_31[col].isnull().sum())
    nan_colT.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listT :
    if col[1] == len(train_T_31) :
        del_col.append(col[0])
train_T_31 = train_T_31.drop(columns=del_col)
test_T_31 = test_T_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_T_31.columns
for col in col_list[6:] :
    if train_T_31[col].nunique()==1 :
        del_col.append(col)
train_T_31 = train_T_31.drop(columns=del_col)
test_T_31 = test_T_31.drop(columns=del_col)

In [ ]:
# drop_col = trainA_31.T[trainA_31.T.duplicated()].index

# trainA_31 = trainA_31.drop(columns=drop_col)
# testA_31 = testA_31.drop(columns=drop_col)

In [ ]:
def random_aug(x):
    while True:
        idx = np.random.randint(6, len(x))
        try:
            int(x[idx])
            break
        except:
            pass
    x[idx] = np.NaN
    # if x[2] == 0: 
    #     x[2] -= np.random.random()*0.01
    # elif x[2] == 2: 
    #     x[2] += np.random.random()*0.01
    return x

trainA_31_aug = trainA_31[trainA_31['Y_Class'] != 1].apply(random_aug, axis=1)
trainA_31 = pd.concat([trainA_31, trainA_31_aug])

train_T_31_aug = train_T_31[train_T_31['Y_Class'] != 1].apply(random_aug, axis=1)
train_T_31 = pd.concat([train_T_31, train_T_31_aug])

train_O_31_aug = train_O_31[train_O_31['Y_Class'] != 1].apply(random_aug, axis=1)
train_O_31 = pd.concat([train_O_31, train_O_31_aug])

In [ ]:
trainA_31_x = trainA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
testA_31_x = testA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
train_T_31_x = train_T_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','PRODUCT_CODE'])
test_T_31_x = test_T_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
train_O_31_x = train_O_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
test_O_31_x = test_O_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])

# classification
trainA_31_y_c = trainA_31['Y_Class']
train_T_31_y_c = train_T_31['Y_Class']
train_O_31_y_c = train_O_31['Y_Class']

# regression
trainA_31_y_r = trainA_31['Y_Quality'].map(np.log1p)
train_T_31_y_r = train_T_31['Y_Quality'].map(np.log1p)
train_O_31_y_r = train_O_31['Y_Quality'].map(np.log1p)

In [ ]:
print(trainA_31_x.shape,'\n',testA_31_x.shape,'\n',train_T_31_x.shape,'\n',test_T_31_x.shape,'\n',train_O_31_x.shape,'\n',test_O_31_x.shape)

(380, 1866) 
 (67, 1866) 
 (401, 550) 
 (239, 550) 
 (8, 499) 
 (4, 499)


In [ ]:
trainA_31_x['LINE'].unique()

array(['T050304', 'T050307', 'T010306', 'T010305'], dtype=object)

In [ ]:
testA_31_x['LINE'].unique()

array(['T010305', 'T010306', 'T050304', 'T050307'], dtype=object)

In [ ]:
trainA_31_x=trainA_31_x.fillna(-1)
testA_31_x=testA_31_x.fillna(-1)
train_T_31_x=train_T_31_x.fillna(-1)
test_T_31_x=test_T_31_x.fillna(-1)
train_O_31_x=train_O_31_x.fillna(-1)
test_O_31_x=test_O_31_x.fillna(-1)

In [ ]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(trainA_31_x[i])
    trainA_31_x[i] = le.transform(trainA_31_x[i])
    
    for label in np.unique(testA_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    testA_31_x[i] = le.transform(testA_31_x[i]) 
print('Done.')

Done.


In [ ]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_T_31_x[i])
    train_T_31_x[i] = le.transform(train_T_31_x[i])
    
    for label in np.unique(test_T_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_T_31_x[i] = le.transform(test_T_31_x[i]) 
print('Done.')

Done.


In [ ]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_O_31_x[i])
    train_O_31_x[i] = le.transform(train_O_31_x[i])
    
    for label in np.unique(test_O_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_O_31_x[i] = le.transform(test_O_31_x[i]) 
print('Done.')

Done.


In [ ]:
#from imblearn.combine import SMOTETomek

In [ ]:
from catboost import *
from lightgbm import *

In [ ]:
model = CatBoostRegressor(random_state=1234,verbose=500,iterations=1500,learning_rate=0.033)
# model = LGBMRegressor(random_state=1234,verbose=500,iterations=1500,learning_rate=0.033)
model.fit(trainA_31_x, trainA_31_y_r)
pred_a = model.predict(testA_31_x)

0:	learn: 0.0072119	total: 375ms	remaining: 9m 22s
500:	learn: 0.0002433	total: 1m 45s	remaining: 3m 30s
1000:	learn: 0.0000324	total: 3m 25s	remaining: 1m 42s
1499:	learn: 0.0000117	total: 5m 3s	remaining: 0us


In [ ]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(train_T_31_x, train_T_31_y_r)
pred_t = model.predict(test_T_31_x)

0:	learn: 0.0038166	total: 140ms	remaining: 3m 29s
500:	learn: 0.0002201	total: 39.6s	remaining: 1m 18s
1000:	learn: 0.0000366	total: 1m 18s	remaining: 39.1s
1499:	learn: 0.0000110	total: 1m 57s	remaining: 0us


In [ ]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(train_O_31_x, train_O_31_y_r)
pred_o = model.predict(test_O_31_x)

0:	learn: 0.0019958	total: 15.6ms	remaining: 23.3s
500:	learn: 0.0000263	total: 6.25s	remaining: 12.5s
1000:	learn: 0.0000004	total: 15.3s	remaining: 7.62s
1499:	learn: 0.0000000	total: 22.8s	remaining: 0us


In [ ]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_quanlity'] = testA_31['Y_quanlity'].map(np.expm1)
test_T_31['Y_quanlity'] = test_T_31['Y_quanlity'].map(np.expm1)
test_O_31['Y_quanlity'] = test_O_31['Y_quanlity'].map(np.expm1)

In [ ]:
testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

In [ ]:
testA_31.loc[(testA_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

test_T_31.loc[(test_T_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

test_O_31.loc[(test_O_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

In [ ]:
submita = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/sample_submission.csv')
submitt = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/sample_submission.csv')
submito = pd.read_csv('/content/drive/MyDrive/스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤/sample_submission.csv')

In [ ]:
submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

In [ ]:
submita['Y_Class'].value_counts()

1    34
0    31
2     2
Name: Y_Class, dtype: int64

In [ ]:
submitt['Y_Class'].value_counts()

1    228
2      6
0      5
Name: Y_Class, dtype: int64

In [ ]:
submito['Y_Class'].value_counts()

1    4
Name: Y_Class, dtype: int64

In [ ]:
submit = pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID')
submit.to_csv('submit.csv',index=False)

In [ ]:
submit['Y_Class'].value_counts()

1    266
0     36
2      8
Name: Y_Class, dtype: int64